# Merging #

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

ets = pd.read_csv('./data/base_etablissement_par_tranche_effectif.csv')
salary = pd.read_csv('./data/net_salary_per_town_categories.csv')
population = pd.read_csv('./data_no_github/population.csv', dtype={'CODGEO': 'string'}) #Pour corriger le dtype qui s'affiche en integer par défaut
geo = pd.read_csv('./data/name_geographic_information.csv', dtype={'code_insee': 'string'})

In [2]:
ets.head()

,CODGEO,LIBGEO,REG,DEP,E14TST,E14TS0ND,E14TS1,E14TS6,E14TS10,E14TS20,E14TS50,E14TS100,E14TS200,E14TS500
0,01001,L'Abergement-Clémenciat,82,01,25,22,1,2,0,0,0,0,0,0
1,01002,L'Abergement-de-Varey,82,01,10,9,1,0,0,0,0,0,0,0
2,01004,Ambérieu-en-Bugey,82,01,996,577,272,63,46,24,9,3,2,0
3,01005,Ambérieux-en-Dombes,82,01,99,73,20,3,1,2,0,0,0,0
4,01006,Ambléon,82,01,4,4,0,0,0,0,0,0,0,0


In [3]:
salary.head()

,CODGEO,LIBGEO,SNHM14,SNHMC14,SNHMP14,SNHME14,SNHMO14,SNHMF14,SNHMFC14,SNHMFP14,SNHMFE14,SNHMFO14,SNHMH14,SNHMHC14,SNHMHP14,SNHMHE14,SNHMHO14,SNHM1814,SNHM2614,SNHM5014,SNHMF1814,SNHMF2614,SNHMF5014,SNHMH1814,SNHMH2614,SNHMH5014
0,01004,Ambérieu-en-Bugey,13.7,24.2,15.5,10.3,11.2,11.6,19.1,13.2,10.1,9.6,15.0,26.4,16.7,11.0,11.6,10.5,13.7,16.1,9.7,11.8,12.5,11.0,14.9,18.6
1,01007,Ambronay,13.5,22.1,14.7,10.7,11.4,11.9,19.0,13.3,10.6,10.0,14.7,23.3,15.8,11.3,11.7,9.8,13.8,14.6,9.2,12.2,12.5,10.2,14.9,16.4
2,01014,Arbent,13.5,27.6,15.6,11.1,11.1,10.9,19.5,11.7,10.8,9.5,15.3,30.2,17.2,12.4,11.8,9.3,13.3,16.0,8.9,10.6,12.5,9.6,15.1,18.6
3,01024,Attignat,12.9,21.8,14.1,11.0,11.3,11.4,19.0,13.0,10.3,9.9,13.8,23.0,14.7,13.2,11.6,9.6,12.9,14.2,9.3,11.4,12.2,9.7,13.8,15.9
4,01025,Bâgé-la-Ville,13.0,22.8,14.1,10.5,11.1,11.6,19.4,13.6,10.2,9.8,13.8,24.1,14.4,11.7,11.4,9.4,12.8,15.2,9.0,11.8,12.3,9.7,13.4,16.9


In [4]:
population.head()

,NIVGEO,CODGEO,LIBGEO,MOCO,AGEQ80_17,SEXE,NB
0,COM,01001,L'Abergement-Clémenciat,11,0,1,15
1,COM,01001,L'Abergement-Clémenciat,11,0,2,15
2,COM,01001,L'Abergement-Clémenciat,11,5,1,20
3,COM,01001,L'Abergement-Clémenciat,11,5,2,20
4,COM,01001,L'Abergement-Clémenciat,11,10,1,20


In [5]:
len(population)

8536584

In [6]:
geo['code_insee_len'] = geo['code_insee'].apply(lambda x: len(x))

In [7]:
#il ya donc 3233 CODGEO avec 4 caractères au lieu de 4. Manque le 0 uniquement ?
geo['code_insee_len'].value_counts()

5    33607
4     3233
Name: code_insee_len, dtype: int64

In [8]:
#Ce ne sont bien que les départements commençant par 0 qui apparaissent dans les codes insee à 4 caractères. C'est donc bien le 0 du code INSEE qui est tronqué
geo[geo['code_insee_len'] == 4]['numéro_département'].value_counts()

02    816
08    464
01    419
07    339
09    332
03    320
04    200
05    177
06    166
Name: numéro_département, dtype: int64

In [9]:
def change_code_insee(x):
    if x['code_insee_len'] == 4:
        return '0'+ x['code_insee']
    else:
        return x['code_insee']
geo['code_insee'] = geo.apply(lambda x: change_code_insee(x), axis=1)

In [10]:
geo = geo.rename(columns={'code_insee': 'CODGEO'})

In [11]:
geo = geo.drop(['code_insee_len'], axis=1)

In [12]:
geo.head()

,EU_circo,code_région,nom_région,chef.lieu_région,numéro_département,nom_département,préfecture,numéro_circonscription,nom_commune,codes_postaux,CODGEO,latitude,longitude,éloignement
0,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Attignat,01340,01024,46.283333,5.166667,1.21
1,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Beaupont,01270,01029,46.400000,5.266667,1.91
2,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Bény,01370,01038,46.333333,5.283333,1.51
3,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Béreyziat,01340,01040,46.366667,5.05,1.71
4,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Bohas-Meyriat-Rignat,01250,01245,46.133333,5.4,1.01


In [13]:
geo.duplicated().value_counts()

False    36840
dtype: int64

In [14]:
merged = geo.merge(ets, how='left', on='CODGEO')

In [15]:
merged2 = merged.merge(salary, how='left', on='CODGEO')

In [16]:
#merged3 = merged2.merge(population, how='left', on='CODGEO')

In [17]:
len(merged2)

36840

In [18]:
merged2.head(50)

,EU_circo,code_région,nom_région,chef.lieu_région,numéro_département,nom_département,préfecture,numéro_circonscription,nom_commune,codes_postaux,CODGEO,latitude,longitude,éloignement,LIBGEO_x,REG,DEP,E14TST,E14TS0ND,E14TS1,E14TS6,E14TS10,E14TS20,E14TS50,E14TS100,E14TS200,E14TS500,LIBGEO_y,SNHM14,SNHMC14,SNHMP14,SNHME14,SNHMO14,SNHMF14,SNHMFC14,SNHMFP14,SNHMFE14,SNHMFO14,SNHMH14,SNHMHC14,SNHMHP14,SNHMHE14,SNHMHO14,SNHM1814,SNHM2614,SNHM5014,SNHMF1814,SNHMF2614,SNHMF5014,SNHMH1814,SNHMH2614,SNHMH5014
0,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Attignat,01340,01024,46.283333,5.166667,1.21,Attignat,82.0,01,116.0,79.0,28.0,4.0,0.0,1.0,1.0,3.0,0.0,0.0,Attignat,12.9,21.8,14.1,11.0,11.3,11.4,19.0,13.0,10.3,9.9,13.8,23.0,14.7,13.2,11.6,9.6,12.9,14.2,9.3,11.4,12.2,9.7,13.8,15.9
1,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Beaupont,01270,01029,46.400000,5.266667,1.91,Beaupont,82.0,01,25.0,19.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Bény,01370,01038,46.333333,5.283333,1.51,Bény,82.0,01,31.0,25.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Béreyziat,01340,01040,46.366667,5.05,1.71,Béréziat,82.0,01,13.0,10.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Bohas-Meyriat-Rignat,01250,01245,46.133333,5.4,1.01,Bohas-Meyriat-Rignat,82.0,01,41.0,34.0,6.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Bourg-en-Bresse,01000,01053,46.200000,5.216667,1.00,Bourg-en-Bresse,82.0,01,3103.0,1722.0,911.0,165.0,159.0,87.0,36.0,13.0,8.0,2.0,Bourg-en-Bresse,12.4,22.8,13.8,10.2,10.3,11.4,20.1,13.1,10.0,8.6,13.1,24.2,14.3,10.7,10.7,9.5,12.2,14.6,9.1,11.2,13.2,9.8,12.9,15.8
6,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Buellas,01310,01065,46.200000,5.133333,0.73,Buellas,82.0,01,68.0,44.0,20.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Certines,01240,01069,46.133333,5.266667,1.97,Certines,82.0,01,57.0,39.0,8.0,1.0,3.0,4.0,2.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Ceyzériat,01250,01072,46.166667,5.316667,1.15,Ceyzériat,82.0,01,188.0,114.0,47.0,9.0,9.0,4.0,5.0,0.0,0.0,0.0,Ceyzériat,14.0,25.0,14.4,10.6,10.8,12.4,21.1,13.4,10.4,9.1,15.2,26.9,15.1,11.4,11.2,9.9,13.9,15.7,9.1,12.5,13.2,10.6,14.8,17.8
9,Sud-Est,82,Rhône-Alpes,Lyon,01,Ain,Bourg-en-Bresse,1,Chavannes-sur-Suran,01250,01095,46.266667,5.416667,1.14,Chavannes-sur-Suran,82.0,01,33.0,28.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
merged2.columns.to_list()

['EU_circo',
 'code_région',
 'nom_région',
 'chef.lieu_région',
 'numéro_département',
 'nom_département',
 'préfecture',
 'numéro_circonscription',
 'nom_commune',
 'codes_postaux',
 'CODGEO',
 'latitude',
 'longitude',
 'éloignement',
 'LIBGEO_x',
 'REG',
 'DEP',
 'E14TST',
 'E14TS0ND',
 'E14TS1',
 'E14TS6',
 'E14TS10',
 'E14TS20',
 'E14TS50',
 'E14TS100',
 'E14TS200',
 'E14TS500',
 'LIBGEO_y',
 'SNHM14',
 'SNHMC14',
 'SNHMP14',
 'SNHME14',
 'SNHMO14',
 'SNHMF14',
 'SNHMFC14',
 'SNHMFP14',
 'SNHMFE14',
 'SNHMFO14',
 'SNHMH14',
 'SNHMHC14',
 'SNHMHP14',
 'SNHMHE14',
 'SNHMHO14',
 'SNHM1814',
 'SNHM2614',
 'SNHM5014',
 'SNHMF1814',
 'SNHMF2614',
 'SNHMF5014',
 'SNHMH1814',
 'SNHMH2614',
 'SNHMH5014']

In [20]:
merged2[pd.isnull(merged2['LIBGEO_x'])][['LIBGEO_x', 'LIBGEO_y']].isna().sum()

LIBGEO_x    394
LIBGEO_y    394
dtype: int64

In [21]:
merged2 = merged2.drop(['LIBGEO_y'], axis=1)


In [22]:
merged2 = merged2.rename(columns={'LIBGEO_x':'LIBGEO'})

In [23]:
import numpy as np
len(np.unique(merged2.CODGEO))

36693

In [24]:
len(merged2)

36840

In [25]:
merged2.duplicated().value_counts()

False    36840
dtype: int64

In [26]:
no_float = []
for i in merged2.longitude:
    try:
        float(i)
    except:
        no_float.append(i)
print(np.unique(no_float))

['-' '-0,75' '-1,94' '-2,24' '-3,43' '-5,1' '0,08' '0,25' '0,41' '0,89'
 '0,94' '0,97' '0,98' '1,02' '1,09' '1,11' '1,12' '1,15' '1,17' '1,18'
 '1,2' '1,21' '1,23' '1,28' '1,29' '1,3' '1,31' '1,38' '1,5' '1,54' '1,63'
 '1,64' '1,67' '1,68' '1,69' '1,71' '1,74' '1,76' '1,78' '1,8' '1,88'
 '1,91' '1,93' '1,95' '1,97' '2,03' '2,07' '2,15' '2,25' '2,43' '2,51'
 '2,55' '2,89' '2,93' '3,01' '3,09' '3,28' '3,47' '3,48' '3,84' '3,89'
 '3,93' '4,07' '4,32' '4,33' '4,76' '4,77' '4,9' '5,04' '5,28' '5,82'
 '5,83' '5,84' '5,96' '6,01' '6,24' '6,31' '6,74']


In [27]:
merged2.longitude = merged2.longitude.replace('-', np.nan)

In [28]:
merged2.longitude = merged2.longitude.str.replace(',', '.')

In [29]:
merged2.longitude = merged2.longitude.astype(float)

In [30]:
merged2.isna().sum()

EU_circo                      0
code_région                   0
nom_région                    0
chef.lieu_région              0
numéro_département            0
nom_département               0
préfecture                    0
numéro_circonscription        0
nom_commune                   0
codes_postaux                 0
CODGEO                        0
latitude                   2929
longitude                  2905
éloignement                2962
LIBGEO                      394
REG                         394
DEP                         394
E14TST                      394
E14TS0ND                    394
E14TS1                      394
E14TS6                      394
E14TS10                     394
E14TS20                     394
E14TS50                     394
E14TS100                    394
E14TS200                    394
E14TS500                    394
SNHM14                    31592
SNHMC14                   31592
SNHMP14                   31592
SNHME14                   31592
SNHMO14 

In [50]:
merged2.E14TST.isna().sum()

0

In [51]:
len(merged2)

36446

In [31]:
merged2 = merged2.dropna(subset=['E14TST']).reset_index(drop=True)

In [52]:
len(merged2)

36446

In [32]:
merged2['ets_categorie'] = pd.qcut(merged2['E14TST'], q=[0,0.25,0.50,0.75,1], labels=['quantile 1', 'quantile 2', 'quantile 3', 'quantile 4'])

In [33]:
merged_ets = merged2[['longitude', 'latitude', 'E14TST', 'ets_categorie']]

In [34]:
def Gcolor(x):
    if x == 'quantile 1':
        return 230
    elif x == 'quantile 2':
        return 190
    elif x == 'quantile 3':
        return 130
    else:
        return 0
merged_ets['Gcolor'] = merged_ets['ets_categorie'].apply(lambda x: Gcolor(x))

C:\Users\vivie\AppData\Local\Temp/ipykernel_42964/1876859912.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_ets['Gcolor'] = merged_ets['ets_categorie'].apply(lambda x: Gcolor(x))


In [35]:
merged_ets = merged_ets.dropna().reset_index(drop=True)

In [36]:
len(merged_ets)

33513

In [37]:
merged_ets.to_csv('merged_ets.csv')

In [38]:
merged_ets['E14TST'].max()

427385.0

In [39]:
merged_ets.head()

,longitude,latitude,E14TST,ets_categorie,Gcolor
0,5.166667,46.283333,116.0,quantile 4,0
1,5.266667,46.400000,25.0,quantile 3,130
2,5.283333,46.333333,31.0,quantile 3,130
3,5.050000,46.366667,13.0,quantile 2,190
4,5.400000,46.133333,41.0,quantile 3,130


In [49]:
merged_ets[merged_ets['ets_categorie'] == 'quantile 1'].E14TST.min()

0.0

In [48]:
merged_ets[merged_ets['ets_categorie'] == 'quantile 1'].E14TST.max()

8.0

In [47]:
merged_ets[merged_ets['ets_categorie'] == 'quantile 2'].E14TST.min()

9.0

In [46]:
merged_ets[merged_ets['ets_categorie'] == 'quantile 2'].E14TST.max()

19.0

In [43]:
merged_ets[merged_ets['ets_categorie'] == 'quantile 3'].E14TST.min()

20.0

In [45]:
merged_ets[merged_ets['ets_categorie'] == 'quantile 3'].E14TST.max()

55.0

In [42]:
merged_ets[merged_ets['ets_categorie'] == 'quantile 4'].E14TST.min()

56.0